In [ ]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:10809"
os.environ["https_proxy"] = "http://127.0.0.1:10809"
from ltp import LTP
ltp = LTP("LTP/small")
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import emoji  

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd
from matplotlib.ticker import MaxNLocator

def parse_date_with_multiple_formats(date_str):
    formats = ['%Y-%m-%d %H:%M:%S', '%Y-%m-%d %H:%M']  # 可以添加更多的格式
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # 如果所有格式都不匹配，返回 NaT

# 假设 all_comments['date'] 是你需要解析的日期字符串列
# all_comments['date'] = all_comments['date'].apply(parse_date_with_multiple_formats)

## Intergrate our data

In [ ]:
# load the mainstream media list
with open('官方媒体/官媒清单.txt', 'r') as f:
    data = f.read()
    data = data.split('\n')
m_media = [item for item in data if item != '']

In [ ]:
import pandas as pd

# Load the comments dataset
topic_comments = pd.read_csv('./data/新冠topic_comments_用户数据/新冠topic_comments.csv')

# Define the types of users according to their verification type
# '1' and '3' represent mainstream media, '0', '200', '220' denote self-media, and '-1' represents ordinary people
topic_comments['type'] = 'None'  # Initialize the column with 'None'
topic_comments.loc[topic_comments['comment_user_verify_type'] == 1, 'type'] = 'm_media'
topic_comments.loc[topic_comments['comment_user_verify_type'] == 3, 'type'] = 'm_media'
topic_comments.loc[topic_comments['comment_user_verify_type'].isin([0, 200, 220]), 'type'] = 'w_media'
topic_comments.loc[topic_comments['comment_user_verify_type'] == -1, 'type'] = 'o_people'

# Overwrite the type if the user name is in the predefined list of mainstream media
topic_comments.loc[topic_comments['comment_user_name'].isin(m_media), 'type'] = 'm_media'

# Extract and clean the list of unique user names for ordinary people
people_name = topic_comments[topic_comments['type'] == 'o_people']['comment_user_name'].unique().tolist()
people_name = [name for name in people_name if name]

# Extract and clean the list of unique user names for self-media
we_media_name = topic_comments[topic_comments['type'] == 'w_media']['comment_user_name'].unique().tolist()
we_media_name = [name for name in we_media_name if name]

# Extract and clean the list of unique user names for mainstream media
m_media_name = topic_comments[topic_comments['type'] == 'm_media']['comment_user_name'].unique().tolist()
m_media_name = [name for name in m_media_name if name]

# Combine the mainstream media names with a predefined list of mainstream media
m_media_name = list(set(m_media_name + m_media))

# Select a sample of relevant columns for the analysis
topic_comments_sample = topic_comments[['comment_user_name', 'comment_time', 'comment_content', 'type']]
topic_comments_sample.columns = ['user_name', 'date', 'content', 'type']

# Display the first few rows of the sample data
topic_comments_sample.head()


In [ ]:
# Load the second dataset
another_comments = pd.read_excel('./output/data.xlsx')

# Initialize the 'type' column
another_comments['type'] = 'None'

# Classify users into 'o_people' if they are not verified and 'w_media' if they are verified
another_comments.loc[another_comments['type'] == '没有认证', 'type'] = 'o_people'
another_comments.loc[another_comments['type'] != '没有认证', 'type'] = 'w_media'

# If the user name is in the predefined list of mainstream media, classify as 'm_media'
another_comments.loc[another_comments['user_name'].isin(m_media), 'type'] = 'm_media'

# Extract and clean the list of unique user names for ordinary people
another_people_name = another_comments[another_comments['type'] == 'o_people']['user_name'].unique().tolist()
another_people_name = [name for name in another_people_name if name]

# Extract and clean the list of unique user names for self-media
another_we_media_name = another_comments[another_comments['type'] == 'w_media']['user_name'].unique().tolist()
another_we_media_name = [name for name in another_we_media_name if name]

# Extract and clean the list of unique user names for mainstream media
another_m_media_name = another_comments[another_comments['type'] == 'm_media']['user_name'].unique().tolist()
another_m_media_name = [name for name in another_m_media_name if name]

# Merge the user names from both datasets to form a complete list
all_people_name = list(set(people_name + another_people_name))
all_we_media_name = list(set(we_media_name + another_we_media_name))
all_m_media_name = list(set(m_media_name + another_m_media_name))

# Select relevant columns for analysis and rename them
another_comments_sample = another_comments[['user_name', 'publish_time', 'content', 'type']]
another_comments_sample.columns = ['user_name', 'date', 'content', 'type']

In [ ]:
# Load the third dataset
repost_comments = pd.read_csv('./data/新冠topic_reposts_关系数据/新冠topic_reposts.csv')

# Initialize the 'type' column to 'None'
repost_comments['type'] = 'None'

# Assign 'o_people' to user types if the user name is found in the list of all people names
repost_comments.loc[repost_comments['user_name'].isin(all_people_name), 'type'] = 'o_people'

# Assign 'w_media' to user types if the user name is found in the list of all self-media names
repost_comments.loc[repost_comments['user_name'].isin(all_we_media_name), 'type'] = 'w_media'

# Assign 'm_media' to user types if the user name is found in the list of all mainstream media names
repost_comments.loc[repost_comments['user_name'].isin(all_m_media_name), 'type'] = 'm_media'

# Count the number of occurrences for each user type
repost_comments['type'].value_counts()

# Filter out comments that have not been categorized into a type
repost_comments = repost_comments[repost_comments['type'] != 'None']

# Select relevant columns for further analysis and rename them
repost_comments_sample = repost_comments[['user_name', 'publish_time', 'content', 'type']]
repost_comments_sample.columns = ['user_name', 'date', 'content', 'type']

In [ ]:
# concat all datasets
all_comments = pd.concat([topic_comments_sample, another_comments_sample, repost_comments_sample], axis=0)
all_comments.drop_duplicates(subset=['user_name', 'date', 'content', 'type'], inplace=True)
all_comments['date'] = all_comments['date'].apply(parse_date_with_multiple_formats)
all_comments['date'] = all_comments['date'].dt.date
all_comments['date'] = pd.to_datetime(all_comments['date'])

In [ ]:
# Ensure that Times New Roman is used as the font for all plot text
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12  # You can adjust the base font size here if needed

# Assuming 'forwarding' DataFrame and 'date' column are already defined
# Create a 'year_month' column for grouping by year and month
all_comments['year_month'] = all_comments['date'].dt.to_period('M')

# Calculate the count of forwarding actions per month
monthly_count = all_comments.groupby('year_month').size()

# Start plotting
plt.figure(figsize=(12, 8))  # Set the figure size for better detail

# Plot data
plt.plot(monthly_count.index.to_timestamp(), monthly_count.values, color='royalblue', marker='o', linestyle='-')

# Improve the formatting of the x-axis to handle dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Adjust interval for less crowded x-axis
plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better readability

# Adding a title and labels with increased font sizes for readability
plt.title('Monthly Post Count Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Post Count', fontsize=14)

# Optionally, set the x-axis to only have a maximum number of date ticks
plt.gca().xaxis.set_major_locator(MaxNLocator(6))

# Remove grid lines for a cleaner look
plt.grid(False)

plt.savefig('./graph/post_count_distribution.png', format='png', dpi=300, bbox_inches='tight')


# Enhancing visual style by removing top and right borders
sns.despine()

# Show the plot
plt.show()

In [ ]:
missing_dates_count = all_comments['date'].isna().sum()
print(f"Number of missing or NaT dates: {missing_dates_count}")

In [ ]:
w_topic = all_comments[all_comments['type'] == 'w_media']
w_topic.to_csv('output/risk_media/w_content_info.csv', index=False)
o_topic = all_comments[all_comments['type'] == 'o_people']
o_topic.to_csv('output/sentiment/o_content_info.csv', index=False)
m_topic = all_comments[all_comments['type'] == 'm_media']
m_topic.to_csv('output/risk_media/m_content_info.csv', index=False)

In [ ]:
m_index = []

for index, row in all_comments.iterrows():
    content = row['content']
    if any(media in str(content) for media in m_media):
        m_index.append(index)

In [ ]:
m_topic_add = all_comments.loc[m_index]
m_topic_all = pd.concat([m_topic, m_topic_add], axis=0)
m_topic_all.drop_duplicates(subset=['user_name', 'date', 'content', 'type'], inplace=True)

In [ ]:
m_topic_all.to_csv('output/risk_media/m_content_info_all.csv', index=False)

In [ ]:
# w_topic = topic_comments_sample[topic_comments_sample['type'] == 'w_media'][['comment_user_name', 'comment_content']]
# w_topic.columns = ['user_name', 'content']
# w_topic.to_excel('output/risk_media/w_content_info.xlsx')
# o_topic = topic_comments_sample[topic_comments_sample['type'] == 'o_people'][['comment_user_name', 'comment_content']]
# o_topic.columns = ['user_name', 'content']
# o_topic.to_excel('output/sentiment/o_content_info.xlsx')
# m_topic = topic_comments_sample[topic_comments_sample['type'] == 'm_media'][['comment_user_name', 'comment_content']]
# m_topic.columns = ['user_name', 'content']
# m_topic.to_excel('output/risk_media/m_content_info.xlsx')
# m_comment_info = pd.read_excel('output/risk_media/m_content_info.xlsx')
# pd.concat([m_topic, m_comment_info], axis=0).to_excel('output/risk_media/m_content_info.xlsx', index=False)

In [ ]:
# comments = pd.read_excel('./output/data.xlsx')
# comments['date'] = pd.to_datetime(comments['publish_time'])
# comments['date'] = comments['date'].dt.date
# comments['date'] = pd.to_datetime(comments['date'])
# comments_sample = comments[('2022-12-01' <= comments['date'])&(comments['date'] <= '2022-12-10')]

## create the sentiment lexicon

In [ ]:
# Define a function to read lines, remove newlines and empty lines
def read_and_clean(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        cleaned_lines = [line.strip() for line in lines if line.strip()]
    return cleaned_lines

# Read and clean lines from the 'high_sentiment.txt' file
words_high = read_and_clean('./output/high_sentiment.txt')

# Read and clean lines from the 'low_sentiment.txt' file
words_low = read_and_clean('./output/low_sentiment.txt')

# Read and clean lines from the 'middle_sentiment.txt' file
words_middle = read_and_clean('./output/middle_sentiment.txt')

# Read and clean lines from the 'sentiment_word.txt' file
words_ini = read_and_clean('./output/sentiment_word.txt')

# Combine words from all sentiment levels and remove duplicates
words_combined = list(set(words_high + words_low + words_middle))

# Add the combined list of words to the LTP for sentiment analysis
ltp.add_words(words_combined)

In [ ]:
# # 自定义词表
# ltp.add_word("汤姆去", freq=2)
# ltp.add_words(["外套", "外衣"], freq=2)

# #  分词 cws、词性 pos、命名实体标注 ner、语义角色标注 srl、依存句法分析 dep、语义依存分析树 sdp、语义依存分析图 sdpg
# output = ltp.pipeline(["还是那句话，不管有没有后遗症，没病不要去得病，自己防护好，清洁做到位，不要去恐慌"], tasks=["cws", "pos"])
# # 使用字典格式作为返回结果
# print(output.cws)  # print(output[0]) / print(output['cws']) # 也可以使用下标访问
# print(output.pos)
# print(output.sdp)
# 使用感知机算法实现的分词、词性和命名实体识别，速度比较快，但是精度略低
# ltp_legacy = LTP("LTP/legacy")
# # cws, pos, ner = ltp.pipeline(["他叫汤姆去拿外衣。"], tasks=["cws", "ner"]).to_tuple() # error: NER 需要 词性标注任务的结果
# cws, pos, ner = ltp.pipeline(["他叫汤姆去拿外衣。"], tasks=["cws", "pos", "ner"]).to_tuple()  # to tuple 可以自动转换为元组格式
# # 使用元组格式作为返回结果
# print(cws, pos, ner)

In [ ]:
# 导入一些语料
# with open('./output/people_content_sample.txt', 'r', encoding='utf-8') as f:
#     lines = f.readlines()
#     lines = [line.strip() for line in lines]
#     yuliao= [line for line in lines if line]
# comments = pd.read_excel('./output/data.xlsx')
# comments['date'] = pd.to_datetime(comments['publish_time'])
# comments['date'] = comments['date'].dt.date
# comments['date'] = pd.to_datetime(comments['date'])
# comments_sample = comments[('2022-12-01' <= comments['date'])&(comments['date'] <= '2022-12-10')]

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer


# # 假定yuliao是您的文本列表，这里用示例文本代替
# texts = yuliao

# # 执行pipeline，获取分词和词性标注结果
# segmented_texts = []  # 用于保存处理后的文本，以便计算TF-IDF
# for text in texts:
#     output = ltp.pipeline([text], tasks=["cws", "pos"])
#     words = output.cws[0]
#     pos_tags = output.pos[0]

#     # 根据词性选择关键词（这里以名词、动词、形容词、人名、组织名和成语为例）
#     selected_words = [word for word, pos in zip(words, pos_tags) if pos in ['n', 'v', 'a', 'nh', 'ni', 'i']]
#     segmented_texts.append(" ".join(selected_words))

# # 计算TF-IDF值
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(segmented_texts)

# # 获取词汇和对应的TF-IDF值，然后打印排名前三的词汇
# feature_names = tfidf_vectorizer.get_feature_names_out()
# key_words = []
# for i, text in enumerate(texts):
#     print(f"Text {i+1}: {text}")
#     tfidf_scores = tfidf_matrix.toarray()[i]
#     word_scores = list(zip(feature_names, tfidf_scores))
#     # 过滤掉TF-IDF值为0的词汇，然后根据TF-IDF值降序排序
#     filtered_word_scores = [(word, score) for word, score in word_scores if score > 0]
#     top_word_scores = sorted(filtered_word_scores, key=lambda x: x[1], reverse=True)[:1]
#     for word, score in top_word_scores:
#         print(f"{word}: {score}")
#         key_words.append(word)
#     # print("\n")



# key_words = list(set(key_words))
# len(key_words)
# 将key_words[500:]写入txt文件
# with open('./output/sentiment/key_words.txt', 'w', encoding='utf-8') as f:
#     for word in key_words[500:]:
#         f.write(word + '\n')
# 根据给定的情感唤醒度分类逻辑和互联网语言特点进行词汇归类

# low = ['惊报', '低烧', '发癫', '抓心', '惆怅', '浪费', '副作用',  '消停', '好使', '逼逼', '不见得', '差不离', '反胃', '杜撰', '是吗', '殆尽', '求求', '担心', '难听','善忘']
# high = ['封杀', '丧心病狂', 'die','谎言', '骗人', '砖家', '灭霸', '混乱', '争论', '痛苦', '夸张', '不行']
# middle = ['适可而止','免得', '保持', '商量', '安全',  '运气', '友好', '团结', '争论']

# # 将这三类添加到low_sentiment.txt, high_sentiment.txt, middle_sentiment.txt中
# with open('./output/sentiment/high_sentiment_seed.txt', 'a', encoding='utf-8') as f:
#     for word in high:
#         f.write(word + '\n')

# with open('./output/sentiment/low_sentiment_seed.txt', 'a', encoding='utf-8') as f:
#     for word in low:
#         f.write(word + '\n')

# with open('./output/sentiment/middle_sentiment_seed.txt', 'a', encoding='utf-8') as f:
#     for word in middle:
#         f.write(word + '\n')

## 情感评分计算

In [ ]:
# import lexicons
with open('./output/sentiment/high_sentiment_full.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    high = [line for line in lines if line]

with open('./output/sentiment/low_sentiment_full.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    low = [line for line in lines if line]

with open('./output/sentiment/middle_sentiment_full.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
    middle = [line for line in lines if line]


high_emotion_words = high
middle_emotion_words = middle
low_emotion_words = low

words = high + low + middle
words = list(set(words))

# add words to ltp
ltp.add_words(words)



negative_words = [
    '没敢', '不是', '不', '没', '无', '非', '莫', '弗', '毋',
    '勿', '未', '否', '别', '休', '無', '不曾', '未必', '没有',
    '不要', '难以', '未曾', '并非', '绝不', '不可'
]

# Dictionary of emotional words and their associated sentiment types
all_emotion_words = {**{word: 'high' for word in high_emotion_words},
                     **{word: 'middle' for word in middle_emotion_words},
                     **{word: 'low' for word in low_emotion_words}}


# Function to analyze the sentiment of a sentence
def analyze_sentiment(sentence):
    # Process the sentence using the LTP pipeline to obtain words and dependencies
    output = ltp.pipeline([sentence], tasks=["cws", "dep"])
    words = output['cws'][0]
    dependencies = output['dep'][0]  # Directly get the result of dependency parsing

    # Initialize emotional intensity counters
    emotion_intensity = {'high': 0, 'middle': 0, 'low': 0}

    # Iterate through each word and its dependency relations
    for i, (word, head_index, relation) in enumerate(zip(words, dependencies['head'], dependencies['label'])):
        head_word = words[head_index - 1] if head_index > 0 else 'ROOT'  # LTP indexing starts from 1

        # Calculate the number of negations affecting the sentiment words
        negation_count = sum(1 for w in words[:i] if w in negative_words)

        if word in all_emotion_words:
            emotion_type = all_emotion_words[word]
            adjusted_intensity = calculate_intensity_adjustment(negation_count)
            emotion_intensity[emotion_type] += adjusted_intensity

    # Determine the sentiment type based on the highest emotional intensity
    if len(set(emotion_intensity.values())) <= 1:
        return 'None', 0

    sentiment_type = max(emotion_intensity, key=emotion_intensity.get)
    return sentiment_type, emotion_intensity[sentiment_type]


# Function to calculate the adjustment in intensity based on negations
def calculate_intensity_adjustment(negation_count):
    # Formula to adjust emotional intensity based on the presence of negation words
    adjusted_intensity = (-1)**negation_count * 0.5 * (1 ** 0.5)  # Assuming the initial intensity of an emotion word is 1
    return adjusted_intensity



# Function to preprocess text by removing HTML tags, hyperlinks, user mentions, hashtags,
# special characters, and specific unwanted patterns
def preprocess_text(text):
    text = emoji.demojize(text, language='zh')  # Convert emojis to text
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    text = re.sub(r'http[s]?://\S+', '', text)  # Remove hyperlinks
    text = re.sub(r'@\w+', '', text)  # Remove user mentions
    text = re.sub(r'#\S+#', '', text)  # Remove hashtags
    text = re.sub(r'[^\w\s，。！？、:]+', '', text)  # Remove special characters but keep Chinese characters and punctuation
    text = re.sub(r'转发原文抱歉，作者已设置仅展示半年内微博，此微博已不可见。', '', text)  # Remove specific unwanted text
    text = re.sub(r'6', '无语', text)  # Replace number 6 with the word for 'speechless'
    text = re.sub(r'\s2\S+$', '', text)  # Remove trailing addresses starting with '2'
    text = re.sub(r'母羊|公羊', '感染', text)  # Replace 'ewe' or 'ram' with 'infection'
    
    return text.strip()  # Trim whitespaces from the beginning and the end

In [ ]:
# import the ordinary people corpse
data = pd.read_csv('./output/sentiment/o_content_info.csv')
# data['content'].to_list()
data['cl_content'] = data['content'].apply(preprocess_text)
data = data[data['cl_content'] != '']
data = data[data['cl_content'] != '转发微博']
data = data.loc[~(data['cl_content'].apply(lambda x: True if '轻症都有盲盒呀，而且年轻人是大多数' in x else False))]
data = data.loc[~(data['cl_content'].apply(lambda x: True if '比比皆是，譬如甲流中招又发烧的就是警示，建议煎几副加减八味肾气汤加味' in x else False))]

In [ ]:
# from tqdm import tqdm
# def process_batch(content_batch):
#     """处理一个数据批次，并返回情感和强度列表"""
#     batch_sentiment = []
#     batch_intensity = []
#     for sentence in content_batch:
#         sent, intens = analyze_sentiment(sentence)
#         batch_sentiment.append(sent)
#         batch_intensity.append(intens)
#     return batch_sentiment, batch_intensity


In [ ]:
# Assuming 'data' is a DataFrame and 'cl_content' is the column with text data
content = data['cl_content'].tolist()

# Replace full-width space character with a regular space
processed_content = [sentence.replace('\u3000', ' ') for sentence in content]

# Initialize lists to store sentiment analysis results
sentiments = []
intensity = []

# Analyze sentiment for each sentence in the content list
for sentence in tqdm(processed_content, desc="Analyzing Sentiments"):
    try:
        sent, intens = analyze_sentiment(sentence)
        sentiments.append(sent)
        intensity.append(intens)
    except Exception as e:
        # Handle exceptions (e.g., errors in sentiment analysis)
        print(f"Error processing sentence: {sentence[:50]}... Error: {e}")
        sentiments.append('None')
        intensity.append(0)

# Add sentiment and intensity results to the original DataFrame
data['sentiment'] = sentiments
data['intensity'] = intensity

# Create a copy of the DataFrame for further modifications
data_copy = data.copy()

# Post-process to refine sentiment classification based on specific content cues
for idx, (content, sentiment) in enumerate(zip(data_copy['cl_content'], data_copy['sentiment'])):
    if sentiment == 'None' and '？' in content:
        data_copy.at[idx, 'sentiment'] = 'low'

In [ ]:
# s_sum = 1383 + 1037 + 677
# high_p = 1383 / s_sum
# middle_p = 1037 / s_sum
# low_p = 677 / s_sum
# print(high_p, middle_p, low_p)

In [ ]:
data_copy.to_csv('./output/sentiment/data_sentiment.csv', index=False)

## Risk appeal

### Mainstream media

In [ ]:
# Read risk-related words from a text file and preprocess them
def read_risk_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        processed_lines = [line.strip() for line in lines if line.strip()]
    return processed_lines

# Load risk and no-risk words
risk_words = read_risk_words('./output/risk_media/risk_words.txt')
no_risk_words = read_risk_words('./output/risk_media/norisk_words.txt')

# Combine and deduplicate the words
all_risk_related_words = list(set(risk_words + no_risk_words))

# Add words to the LTP's vocabulary
ltp.add_words(all_risk_related_words)

# Create a dictionary to classify words as 'yes' for risk or 'no' for no-risk
all_risk_words = {**{word: 'yes' for word in risk_words}, **{word: 'no' for word in no_risk_words}}

# List of negation words that might alter the context of risk words
negative_words = [
    '没敢', '不是', '不', '没', '无', '非', '莫', '弗', '毋',
    '勿', '未', '否', '别', '休', '无', '不曾', '未必', '没有',
    '不要', '难以', '未曾', '并非', '绝不', '不可'
]

# Function to analyze the risk in a sentence using LTP
def analyze_risk(sentence):
    # Process the sentence with LTP for word segmentation and dependency parsing
    output = ltp.pipeline([sentence], tasks=["cws", "dep"])
    words = output['cws'][0]
    dependencies = output['dep'][0]

    risk_intensity = {'yes': 0, 'no': 0}

    # Analyze each word and its dependencies to assess risk
    for i, (word, head_index, relation) in enumerate(zip(words, dependencies['head'], dependencies['label'])):
        head_word = words[head_index - 1] if head_index > 0 else 'ROOT'  # LTP's head indices start from 1

        # Check if a negation word directly modifies a risk word
        negation_count = sum(1 for w in words[:i] if w in negative_words and words[dependencies['head'][i] - 1] in all_risk_words)

        if word in all_risk_words:
            risk_type = all_risk_words[word]
            adjusted_intensity = calculate_intensity_adjustment(negation_count)
            risk_intensity[risk_type] += adjusted_intensity

    # Determine the predominant risk category
    if len(set(risk_intensity.values())) <= 1:
        return 'None', 0  # If all intensities are equal or zero, categorize as 'None'

    risk_type = max(risk_intensity, key=risk_intensity.get)
    return risk_type, risk_intensity[risk_type]

# Function to adjust intensity based on the presence of negation words
def calculate_intensity_adjustment(negation_count):
    # Adjust the intensity based on negation; example formula below
    return (-1) ** negation_count * 0.5 * (1 ** 0.5)  # Assuming initial intensity is 1


In [ ]:
m_content_info = pd.read_csv('./output/risk_media/m_content_info.csv')
# media = data[~(data['verify_typ'] == '没有认证')]
m_content_info['cl_content'] = m_content_info['content'].apply(lambda x: preprocess_text(x))
m_content_info = m_content_info[m_content_info['cl_content'] != '']

In [ ]:
data = m_content_info.copy()
data['cl_content'] = data['cl_content'].apply(lambda x: str(x))
data = data[~(data['cl_content'].apply(lambda x: True if '并不是所有的口干都是上火' in x else False))]
data.reset_index(drop=True, inplace=True)
no_index = []
yes_index = []
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '转发原文七成半新冠康复者受后遗症困扰' in x else False))].index))
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '症状可持续数周、数月甚至更长时间10月13日' in x else False))].index))
content = data['cl_content'].tolist()

risks = []
intensity = []

for i, sentence in tqdm(enumerate(content), desc="分析风险", total=len(content)):
    if i not in yes_index:
        risk, intens = analyze_risk(sentence)
        risks.append(risk)
        intensity.append(intens)
    else:
        risks.append('yes')
        intensity.append(0)
for id in yes_index:
    risks[id] = 'yes'
    intensity[id] = 0
data['risk'] = risks
data.loc[data['content'].apply(lambda x: True if '崇雨田' in x else False), 'risk'] = 'no'
data.loc[data['content'].apply(lambda x: True if '梁连春' in x else False), 'risk'] = 'no'
data.loc[data['content'].apply(lambda x: True if '美国研究' in x else False), 'risk'] = 'yes'
data.loc[data['content'].apply(lambda x: True if '牛津大学' in x else False), 'risk'] = 'yes'

In [ ]:
data.to_csv('./output/risk_media/data_risk_mainstream.csv', index=False)

### We Media

In [ ]:
data = pd.read_csv('./output/risk_media/w_content_info.csv')
data['cl_content'] = data['content'].apply(lambda x: str(x))
data = data[~(data['cl_content'].apply(lambda x: True if '并不是所有的口干都是上火' in x else False))]
data['cl_content'] = data['cl_content'].apply(lambda x: preprocess_text(x))
data = data[data['cl_content'] != '']
data = data.reset_index(drop=True)
no_index = []
yes_index = []
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '中国疾控中心流行病学首席专家吴尊友表示' in x else False))].index))
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '转发原文七成半新冠康复者受后遗症困扰' in x else False))].index))
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '症状可持续数周、数月甚至更长时间10月13日' in x else False))].index))
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '在国务院联防联控机制新闻发布会上，中国疾控中心流行病学首席专家' in x else False))].index))
yes_index.extend(list(data[(data['cl_content'].apply(lambda x: True if '吴尊友' in x else False))].index))
yes_index = list(set(yes_index))

In [ ]:

content = data['cl_content'].tolist()

risks = []
intensity = []

for i, sentence in tqdm(enumerate(content), desc="分析风险", total=len(content)):
    if i not in yes_index:
        risk, intens = analyze_risk(sentence)
        risks.append(risk)
        intensity.append(intens)
    else:
        risks.append('yes')
        intensity.append(0)

In [ ]:
data['risk'] = risks
data['risk'].value_counts()
data.loc[data['content'].apply(lambda x: True if '崇雨田' in str(x) else False), 'risk'] = 'no'
data.loc[data['content'].apply(lambda x: True if '梁连春' in str(x) else False), 'risk'] = 'no'
data.loc[data['content'].apply(lambda x: True if '美国研究' in str(x) else False), 'risk'] = 'yes'
data.loc[data['content'].apply(lambda x: True if '牛津大学' in str(x) else False), 'risk'] = 'yes'


data.loc[data['content'].apply(lambda x: True if '能有什么后遗症' in str(x) else False), 'risk'] = 'no'
data.loc[data['content'].apply(lambda x: True if '白肺患者临床' in str(x) else False), 'risk'] = 'no'
data.loc[data['content'].apply(lambda x: True if '都会有后遗症' in str(x) else False), 'risk'] = 'yes'
data.loc[data['content'].apply(lambda x: True if '心率不齐' in str(x) else False), 'risk'] = 'yes'
data.loc[data['content'].apply(lambda x: True if '非高危组重症为0' in str(x) else False), 'risk'] = 'yes'

data['risk'].value_counts()

In [ ]:
data.to_csv('./output/risk_media/data_risk_wemedia.csv', index=False)

## Calculate the empirical data

In [ ]:
data_sentiment = pd.read_csv('./output/sentiment/data_sentiment.csv')

In [ ]:
# list(data_sentiment[data_sentiment['sentiment'].isna()]['cl_content'].value_counts().index)
# with open('./sentiment_na_content_cl.txt', 'w', encoding='utf-8') as f:
#     for item in list(data_sentiment[data_sentiment['sentiment'].isna()]['cl_content'].value_counts().index):
#         f.write(item + '\n')

In [ ]:
data_risk_mainstream = pd.read_csv('./output/risk_media/data_risk_mainstream.csv')
missing_dates_count = data_risk_mainstream['date'].isna().sum()
print(f"Number of missing or NaT dates: {missing_dates_count}")
data_risk_wemedia = pd.read_csv('./output/risk_media/data_risk_wemedia.csv')
missing_dates_count = data_risk_wemedia['date'].isna().sum()
print(f"Number of missing or NaT dates: {missing_dates_count}")

In [ ]:
# no_risk = ['自限性疾病','成天测','24小时核酸', '能不检就不检']
risk = ['做好防护', '持久战','美国经验','新冠长期症状','共存派','经常记不起', 
        '共蠢派', '长期后遗症影响', '焦虑素材', '新冠后遗症受到越来越多的关注', '胸疼', 
        '阴性对照组有后遗症', '很严重', '也后遗症', '任何后遗症',
        '心惊','柳叶刀','新京报']

# 更新风险列
for index, row in data_risk_wemedia.iterrows():
    content = row['cl_content']
    # if any(word in str(content) for word in no_risk):
    #     data_risk_wemedia.at[index, 'risk'] = 'no'
    if any(word in str(content) for word in risk):
        data_risk_wemedia.at[index, 'risk'] = 'yes'

In [ ]:
# Convert the 'date' column to datetime format
data_risk_wemedia['date'] = pd.to_datetime(data_risk_wemedia['date'])

# Group the data by 'date' and count occurrences of each 'risk' value
data_risk_wemedia = data_risk_wemedia.groupby('date')['risk'].value_counts().unstack().fillna(0)

# Add a 'total' column summing up all risk occurrences per day
data_risk_wemedia['total'] = data_risk_wemedia.sum(axis=1)

# Calculate the proportion of 'yes' and 'no' risk responses
data_risk_wemedia['risk_p'] = data_risk_wemedia['yes'] / data_risk_wemedia['total']
data_risk_wemedia['norisk_p'] = data_risk_wemedia['no'] / data_risk_wemedia['total']

# Reset index to turn the grouped 'date' index back into a column
data_risk_wemedia = data_risk_wemedia.reset_index()

# Filter data within a specific date range
data_risk_wemedia = data_risk_wemedia[
    (data_risk_wemedia['date'] >= '2022-10-14') & 
    (data_risk_wemedia['date'] <= '2023-01-18')
]

# Calculate the number of days since the start date of the dataset
df = data_risk_wemedia.copy()
start_date = df['date'].min()
df['days_since_start'] = (df['date'] - start_date).dt.days

# Create a new period identifier by dividing 'days_since_start' by 7 and taking the floor
df['period_id_7d'] = df['days_since_start'] // 7

# Group the data by the new 7-day period identifier and aggregate values
period_data_7d = df.groupby('period_id_7d').agg(
    start_date=('date', 'min'),  # Minimum date in each period
    yes_total=('yes', 'sum'),   # Sum of 'yes' labels
    no_total=('no', 'sum'),     # Sum of 'no' labels
    total_sum=('total', 'sum')  # Sum of all responses
).reset_index()

# Calculate the percentage of risk and no-risk responses per period
period_data_7d['risk_p'] = period_data_7d['yes_total'] / period_data_7d['total_sum']
period_data_7d['norisk_p'] = period_data_7d['no_total'] / period_data_7d['total_sum']

# Save the aggregated data to a CSV file
period_data_7d.to_csv('./output/empirical/w_media_risk_data_7days.csv', index=False)


In [ ]:
data_risk_mainstream['date'] = pd.to_datetime(data_risk_mainstream['date'])
data_risk_mainstream['date'] = pd.to_datetime(data_risk_mainstream['date'])
date_risk_mainstream =  data_risk_mainstream.groupby('date')['risk'].value_counts().unstack().fillna(0)
date_risk_mainstream['total'] = date_risk_mainstream.sum(axis=1)

# date_risk_mainstream.to_csv('./date_risk_mainstream.csv')
date_risk_mainstream['risk_p'] = date_risk_mainstream['yes'] / date_risk_mainstream['total']
date_risk_mainstream['norisk_p'] = date_risk_mainstream['no'] / date_risk_mainstream['total']
date_risk_mainstream = date_risk_mainstream.reset_index()
date_risk_mainstream = date_risk_mainstream[(date_risk_mainstream['date'] >= '2022-10-14') & (date_risk_mainstream['date'] <= '2023-01-18')]
date_risk_mainstream

df = date_risk_mainstream.copy()
start_date = df['date'].min()
df['days_since_start'] = (df['date'] - start_date).dt.days


df['period_id_7d'] = df['days_since_start'] // 7


period_data_7d = df.groupby('period_id_7d').agg(
    start_date=('date', 'min'),  
    yes_total=('yes', 'sum'),
    no_total=('no', 'sum'),
    # middle_total=('middle', 'sum'),
    total_sum=('total', 'sum')
).reset_index()


period_data_7d['risk_p'] = period_data_7d['yes_total'] / period_data_7d['total_sum']
period_data_7d['norisk_p'] = period_data_7d['no_total'] / period_data_7d['total_sum']

period_data_7d.to_csv('./output/empirical/m_media_risk_data_7days.csv', index=False)

In [ ]:
data_sentiment = pd.read_csv('./output/sentiment/data_sentiment.csv')
middle_words = [
 "戒烟戒酒", "实事求是",
    '你信就是了','矫枉过正','比以前棒','会好点','最有价值','很有水平','不矛盾','还好', '选择信人品', '阿航', '照顾好自己', '多喝热水', '大实话'
]
high_words = ['他们都骂']
low_words = ['我怕', '大明白', '啥都不懂', '很无语']

for index, row in data_sentiment.iterrows():
    content = row['cl_content']
    if any(word in content for word in middle_words):
        data_sentiment.at[index, 'sentiment'] = 'middle'
    elif any(word in content for word in high_words):
        data_sentiment.at[index, 'sentiment'] = 'high'
    elif any(word in content for word in low_words):
        data_sentiment.at[index, 'sentiment'] = 'low'

In [ ]:
data_sentiment['date'] = pd.to_datetime(data_sentiment['date'])
data_sentiment.groupby('date')['sentiment'].value_counts().unstack().fillna(0)

date_sentiment = data_sentiment.groupby('date')['sentiment'].value_counts().unstack().fillna(0)
date_sentiment['total'] = date_sentiment.sum(axis=1)
date_sentiment['high_p'] = date_sentiment['high'] / date_sentiment['total']
date_sentiment['middle_p'] = date_sentiment['middle'] / date_sentiment['total']
date_sentiment['low_p'] = date_sentiment['low'] / date_sentiment['total']
date_sentiment = date_sentiment.reset_index()
date_sentiment_sample = date_sentiment[(date_sentiment['date'] >= '2022-10-14') & (date_sentiment['date'] <= '2023-01-18')]
date_sentiment_sample

In [ ]:

df = date_sentiment_sample.copy()
start_date = df['date'].min()
df['days_since_start'] = (df['date'] - start_date).dt.days


df['period_id_3d'] = df['days_since_start'] // 7


period_data_7d = df.groupby('period_id_3d').agg(
    start_date=('date', 'min'),  
    high_total=('high', 'sum'),
    low_total=('low', 'sum'),
    middle_total=('middle', 'sum'),
    total_sum=('total', 'sum')
).reset_index()


period_data_7d['high_p'] = period_data_7d['high_total'] / period_data_7d['total_sum']
period_data_7d['middle_p'] = period_data_7d['middle_total'] / period_data_7d['total_sum']
period_data_7d['low_p'] = period_data_7d['low_total'] / period_data_7d['total_sum']


period_data_7d.to_csv('./output/empirical/sentiment_data_7days.csv', index=False)

In [ ]:
# # 绘制两幅图，分别是high_p, middle_p, low_p的时间序列图和high, middle, low的时间序列图
# import matplotlib.pyplot as plt

# fig, ax = plt.subplots(2, 1, figsize=(12, 8))

# period_data_7d.plot(x='period_id_7d', y=['high_p', 'middle_p', 'low_p'], ax=ax[0], title='Sentiment Proportion Over Time', color=['r', 'g', 'b'], linestyle='-', linewidth=0.5)
# # period_data_3d.plot(x='period_id_3d', y=['high_total', 'middle_total', 'low_total'], ax=ax[1], title='Sentiment Count Over Time', color=['r', 'g', 'b'],  linestyle='-', linewidth=0.5)

# plt.show()
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np

# # 假设的平滑窗口大小
# window_size = 3

# # 计算滚动平均以平滑数据
# period_data_7d['high_p_smooth'] = period_data_7d['high_p'].rolling(window=window_size, min_periods=1).mean()
# period_data_7d['middle_p_smooth'] = period_data_7d['middle_p'].rolling(window=window_size, min_periods=1).mean()
# period_data_7d['low_p_smooth'] = period_data_7d['low_p'].rolling(window=window_size, min_periods=1).mean()

# # 绘制子图
# fig, axs = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# # 配置标题和轴标签
# titles = ['High Sentiment Proportion', 'Middle Sentiment Proportion', 'Low Sentiment Proportion']
# colors = ['red', 'green', 'blue']
# smooth_labels = ['high_p_smooth', 'middle_p_smooth', 'low_p_smooth']
# original_labels = ['high_p', 'middle_p', 'low_p']

# for i, ax in enumerate(axs):
#     period_data_7d.plot(x='period_id_3d', y=original_labels[i], ax=ax, linestyle='-', color=colors[i], alpha=0.5, linewidth=1, label='Original')
#     period_data_7d.plot(x='period_id_3d', y=smooth_labels[i], ax=ax, linestyle='-', color=colors[i], linewidth=2, label='Smoothed')
#     ax.set_title(titles[i])
#     ax.set_xlabel('Period')
#     ax.set_ylabel('Proportion')
#     ax.legend()

# plt.tight_layout()
# plt.show()
